## 1. Consolidated files in the unique DataFrame and show the total files extracted

In [53]:
import os
import sys

notebook_path = os.path.dirname(os.path.abspath("data_explorer.ipynb"))
sys.path.append(os.path.dirname(notebook_path))

from etl.utils.common import DefaultOutputFolder as dir
import pandas as pd

files = os.listdir(dir())
dfs = []

if not files: print("No files found in the output folder.")

for file in files:
    if file.endswith(".parquet"):
        df = pd.read_parquet(dir() + file)
        dfs.append(df)
    
allFiles = pd.concat(dfs, ignore_index=True)

# Ordering DataFrame by column name
allFiles = allFiles.sort_values(by=['extracted_at'], ascending=False)

# count the rows in dataframe
allFiles.shape[0]

210

## 1.1 Data set sample, list 5 files

In [57]:
allFiles.head(3)

,code,codein,name,high,low,varBid,pctChange,bid,ask,timestamp,create_date,symbol,extracted_at
105,BRL,MAD,Real Brasileiro/Dirham Marroquino,1.9569,1.9213,0.0204,1.05,1.9489,1.9502,1713560398,2024-04-19 17:59:58,BRL-MAD,2024-04-22 01:02:43
68,BRL,JPY,Real Brasileiro/Iene Japonês,29.585,29.549,0.0103,0.03,29.565,29.581,1713747731,2024-04-21 22:02:11,BRL-JPY,2024-04-22 01:02:43
138,BRL,SAR,Real Brasileiro/Riyal Saudita,0.7204,0.7202,-0.0001,-0.01,0.7201,0.7203,1713739283,2024-04-21 19:41:23,BRL-SAR,2024-04-22 01:02:43


## 2. Change DataTypes and Reorder columns

In [56]:
## Reorder columns in dataframe
df = allFiles[['create_date', 'extracted_at', 'code', 'symbol', 'name', 'ask', 'bid', 'varBid', 'pctChange' ]]

# Change data types
df = df.astype({'ask': float, 'bid': float, 'varBid': float, 'pctChange': float})

# Filter dataframe by code
df = df[df['code'] == 'BRL']

# Show the dataframe
df.head(3)


,create_date,extracted_at,code,symbol,name,ask,bid,varBid,pctChange
105,2024-04-19 17:59:58,2024-04-22 01:02:43,BRL,BRL-MAD,Real Brasileiro/Dirham Marroquino,1.9502,1.9489,0.0204,1.05
68,2024-04-21 22:02:11,2024-04-22 01:02:43,BRL,BRL-JPY,Real Brasileiro/Iene Japonês,29.5810,29.5650,0.0103,0.03
138,2024-04-21 19:41:23,2024-04-22 01:02:43,BRL,BRL-SAR,Real Brasileiro/Riyal Saudita,0.7203,0.7201,-0.0001,-0.01
